In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [236]:
game_request = requests.get('https://www.basketball-reference.com/boxscores/pbp/202008250DEN.html')

In [237]:
game_request

<Response [200]>

In [238]:
game_soup = BeautifulSoup(game_request.text, 'lxml')

In [239]:
game_soup

<!DOCTYPE html>
<html class="no-js" data-root="/home/bbr/build" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/202008171" rel="dns-prefetch"/>
<title>Utah Jazz at Denver Nuggets Play-By-Play, August 25, 2020 | Basketball-Reference.com</title>
<meta content="Play-By-Play - Utah Jazz (107) vs. Denver Nuggets (117) - August 25, 2020" name="Description"/>
<link href="https://www.basketball-reference.com/boxscores/pbp/202008250DEN.html" rel="canonical"/>
<!-- include:start ="/inc/klecko_header_bbr.html_f" -->
<!-- no:cookie fast load the css.           -->
<script>function gup(n) {n = n.replace(/[\[]/, '\\[').replace(/[\]]/, '\\]'); var r = new RegExp('[\\?&]'+n+'=([^&#]*)'); var re = r.exec(location.search);   return re === n

In [240]:
play_by_play = game_soup.find_all('tr')
play_by_play

[<tr class="thead" id="q1">
 <th colspan="6">1st Q</th>
 </tr>, <tr class="thead">
 <th aria-label="Time" class="center" data-stat="Time">Time</th>
 <th aria-label="Utah" class="center" data-stat="Utah">Utah</th>
 <th aria-label="" class="center" data-stat=""></th>
 <th aria-label="Score" class="center" data-stat="Score">Score</th>
 <th aria-label="" class="center" data-stat=""></th>
 <th aria-label="Denver" class="center" data-stat="Denver">Denver</th>
 </tr>, <tr>
 <td>12:00.0</td>
 <td class="center" colspan="5">Jump ball: <a href="/players/g/goberru01.html">R. Gobert</a> vs. <a href="/players/j/jokicni01.html">N. Jokić</a> (<a href="/players/m/millspa01.html">P. Millsap</a> gains possession)</td>
 </tr>, <tr>
 <td>11:46.0</td>
 <td> </td><td> </td><td class="center">0-0</td><td class=""> </td><td class=""><a href="/players/m/murraja01.html">J. Murray</a> misses 3-pt jump shot from 27 ft</td>
 </tr>, <tr>
 <td>11:42.0</td>
 <td class="">Defensive rebound by <a href="/players/i/ingle

In [241]:
len(play_by_play[3].contents)

9

In [242]:
len(play_by_play[5].contents)

9

In [243]:
for val in play_by_play:
    if len(val.contents) < 9:
        print(val)

<tr class="thead" id="q1">
<th colspan="6">1st Q</th>
</tr>
<tr>
<td>12:00.0</td>
<td class="center" colspan="5">Jump ball: <a href="/players/g/goberru01.html">R. Gobert</a> vs. <a href="/players/j/jokicni01.html">N. Jokić</a> (<a href="/players/m/millspa01.html">P. Millsap</a> gains possession)</td>
</tr>
<tr>
<td>0:00.0</td>
<td class="center" colspan="5">End of 1st quarter</td>
</tr>
<tr class="thead" id="q2">
<th colspan="6">2nd Q</th>
</tr>
<tr>
<td>12:00.0</td>
<td class="center" colspan="5">Start of 2nd quarter</td>
</tr>
<tr>
<td>0:00.0</td>
<td class="center" colspan="5">End of 2nd quarter</td>
</tr>
<tr class="thead" id="q3">
<th colspan="6">3rd Q</th>
</tr>
<tr>
<td>12:00.0</td>
<td class="center" colspan="5">Start of 3rd quarter</td>
</tr>
<tr>
<td>0:00.0</td>
<td class="center" colspan="5">End of 3rd quarter</td>
</tr>
<tr class="thead" id="q4">
<th colspan="6">4th Q</th>
</tr>
<tr>
<td>12:00.0</td>
<td class="center" colspan="5">Start of 4th quarter</td>
</tr>
<tr>
<td>0:

In [244]:
play_by_play[10].contents[7]

<td class="">Defensive rebound by <a href="/players/m/murraja01.html">J. Murray</a></td>

In [245]:
# Creating function for finding main player / secondary player

def main_secondary_players(content_val):
    players = content_val.find_all()
    
    if len(players) == 0:
        return None, None
    
    # If there's just one player in focus
    elif len(players) == 1:
        return players[0].attrs['href'], None
    
    else:
        return players[0].attrs['href'], players[1].attrs['href']

In [246]:
content_val = play_by_play[5].contents[3]
print(content_val)
players = content_val.find_all()

main_secondary_players(content_val)

<td class="bbr-play-score"><a href="/players/g/goberru01.html">R. Gobert</a> makes 2-pt dunk from 4 ft (assist by <a href="/players/m/mitchdo01.html">D. Mitchell</a>)</td>


('/players/g/goberru01.html', '/players/m/mitchdo01.html')

In [247]:
all_data = []

for line in play_by_play:
    content_information = line.contents
    if len(content_information) == 3:
        period = content_information[1].text
        #print(period)
        continue
    elif len(content_information) == 9:
        time_remaining = content_information[1].text.strip()
        
        visiting_text = content_information[3].text.strip()
        
        visiting_points = content_information[4].text.strip()
        
        visiting_score = content_information[5].text.split('-')[0]
        home_score = content_information[5].text.split('-')[1]
        
        home_points = content_information[6].text.strip()
        home_text = content_information[7].text.strip()
        
        
        if home_text != '':
            main_player, secondary_player = main_secondary_players(content_information[7])
        
        else:
            main_player, secondary_player = main_secondary_players(content_information[3])
        
        
        data = [period, time_remaining, visiting_text, visiting_points, visiting_score, 
                home_score, home_points, home_text,
                main_player, secondary_player]
        
        all_data.append(data)

In [248]:
df_columns = ['Period','Time','Visiting Posession','Visiting Points','Visiting Score',
              'Home Score','Home Points','Home Posession',
              'Main Player','Secondary Player']
df = pd.DataFrame(all_data, columns = df_columns)
df

,Period,Time,Visiting Posession,Visiting Points,Visiting Score,Home Score,Home Points,Home Posession,Main Player,Secondary Player
0,1st Q,11:46.0,,,0,0,,J. Murray misses 3-pt jump shot from 27 ft,/players/m/murraja01.html,None
1,1st Q,11:42.0,Defensive rebound by J. Ingles,,0,0,,,/players/i/inglejo01.html,None
2,1st Q,11:33.0,R. Gobert makes 2-pt dunk from 4 ft (assist by...,+2,2,0,,,/players/g/goberru01.html,/players/m/mitchdo01.html
3,1st Q,11:17.0,,,2,0,,Shooting foul by D. Mitchell (drawn by P. Mill...,/players/m/mitchdo01.html,/players/m/millspa01.html
4,1st Q,11:17.0,,,2,1,+1,P. Millsap makes free throw 1 of 2,/players/m/millspa01.html,None
...,...,...,...,...,...,...,...,...,...,...
400,4th Q,0:13.0,J. Clarkson enters the game for R. O'Neale,,107,116,,,/players/c/clarkjo01.html,/players/o/onealro01.html
401,4th Q,0:13.0,,,107,116,,T. Craig enters the game for M. Porter,/players/c/craigto01.html,/players/p/portemi01.html
402,4th Q,0:13.0,,,107,117,+1,J. Grant makes free throw 2 of 2,/players/g/grantje01.html,None
403,4th Q,0:08.0,D. Mitchell misses 3-pt jump shot from 28 ft,,107,117,,,/players/m/mitchdo01.html,None


In [223]:
# Home Action
home_team_data = df[df['Home Posession'] != ''][['Period','Time','Home Posession','Home Points',
                                    'Home Score','Main Player','Secondary Player']]

# Visiting Action
visiting_team_data = df[df['Visiting Posession'] != ''][['Period','Time','Visiting Posession','Visiting Points',
                                    'Visiting Score','Main Player','Secondary Player']]

In [231]:
full_columns = ['Period','Time','Posession','Points','Score','Main Player','Secondary Player','Team']

In [232]:
home_team_data['Team'] = 'Home'
home_team_data.columns = full_columns
home_team_data.head()

,Period,Time,Posession,Points,Score,Main Player,Secondary Player,Team
1,1st Q,11:35.0,Turnover by S. Hill (bad pass; steal by E. Mud...,,0,/players/h/hillso01.html,/players/m/mudiaem01.html,Home
6,1st Q,11:11.0,E. Moore makes 2-pt jump shot from 8 ft (assis...,+2,2,/players/m/mooreet01.html,/players/d/davisan02.html,Home
8,1st Q,10:41.0,A. Davis misses 2-pt jump shot from 21 ft,,2,/players/d/davisan02.html,None,Home
11,1st Q,10:32.0,Defensive rebound by Team,,2,None,None,Home
13,1st Q,10:11.0,Turnover by E. Moore (bad pass; steal by D. Ga...,,2,/players/m/mooreet01.html,/players/g/gallida01.html,Home


In [233]:
visiting_team_data['Team'] = 'Away'
visiting_team_data.columns = full_columns
visiting_team_data.head()

,Period,Time,Posession,Points,Score,Main Player,Secondary Player,Team
0,1st Q,11:39.0,Turnover by N. Jokić (bad pass; steal by A. Da...,,0,/players/j/jokicni01.html,/players/d/davisan02.html,Away
2,1st Q,11:28.0,Shooting foul by Ö. Aşık (drawn by J. Nurkić),,0,/players/a/asikom01.html,/players/n/nurkiju01.html,Away
3,1st Q,11:28.0,J. Nurkić misses free throw 1 of 2,,0,/players/n/nurkiju01.html,None,Away
4,1st Q,11:28.0,Offensive rebound by Team,,0,None,None,Away
5,1st Q,11:28.0,J. Nurkić makes free throw 2 of 2,+1,1,/players/n/nurkiju01.html,None,Away


In [252]:
full_df = home_team_data.append(visiting_team_data).sort_index()
full_df

,Period,Time,Posession,Points,Score,Main Player,Secondary Player,Team
0,1st Q,11:39.0,Turnover by N. Jokić (bad pass; steal by A. Da...,,0,/players/j/jokicni01.html,/players/d/davisan02.html,Away
1,1st Q,11:35.0,Turnover by S. Hill (bad pass; steal by E. Mud...,,0,/players/h/hillso01.html,/players/m/mudiaem01.html,Home
2,1st Q,11:28.0,Shooting foul by Ö. Aşık (drawn by J. Nurkić),,0,/players/a/asikom01.html,/players/n/nurkiju01.html,Away
3,1st Q,11:28.0,J. Nurkić misses free throw 1 of 2,,0,/players/n/nurkiju01.html,None,Away
4,1st Q,11:28.0,Offensive rebound by Team,,0,None,None,Away
...,...,...,...,...,...,...,...,...
491,4th Q,0:08.0,Defensive rebound by E. Mudiay,,105,/players/m/mudiaem01.html,None,Away
492,4th Q,0:08.0,Personal take foul by E. Moore (drawn by E. Mu...,,105,/players/m/mooreet01.html,/players/m/mudiaem01.html,Away
493,4th Q,0:08.0,E. Mudiay makes free throw 1 of 2,+1,106,/players/m/mudiaem01.html,None,Away
494,4th Q,0:08.0,E. Mudiay makes free throw 2 of 2,+1,107,/players/m/mudiaem01.html,None,Away


In [251]:
"Series Summary" in game_request.text

True

In [256]:
full_df['playoff_game'] = 1 if "Series Summary" in game_request.text else 0 
full_df

,Period,Time,Posession,Points,Score,Main Player,Secondary Player,Team,playoff_game
0,1st Q,11:39.0,Turnover by N. Jokić (bad pass; steal by A. Da...,,0,/players/j/jokicni01.html,/players/d/davisan02.html,Away,1
1,1st Q,11:35.0,Turnover by S. Hill (bad pass; steal by E. Mud...,,0,/players/h/hillso01.html,/players/m/mudiaem01.html,Home,1
2,1st Q,11:28.0,Shooting foul by Ö. Aşık (drawn by J. Nurkić),,0,/players/a/asikom01.html,/players/n/nurkiju01.html,Away,1
3,1st Q,11:28.0,J. Nurkić misses free throw 1 of 2,,0,/players/n/nurkiju01.html,None,Away,1
4,1st Q,11:28.0,Offensive rebound by Team,,0,None,None,Away,1
...,...,...,...,...,...,...,...,...,...
491,4th Q,0:08.0,Defensive rebound by E. Mudiay,,105,/players/m/mudiaem01.html,None,Away,1
492,4th Q,0:08.0,Personal take foul by E. Moore (drawn by E. Mu...,,105,/players/m/mooreet01.html,/players/m/mudiaem01.html,Away,1
493,4th Q,0:08.0,E. Mudiay makes free throw 1 of 2,+1,106,/players/m/mudiaem01.html,None,Away,1
494,4th Q,0:08.0,E. Mudiay makes free throw 2 of 2,+1,107,/players/m/mudiaem01.html,None,Away,1


In [260]:
full_df['Main Player'].apply(lambda x: 1 if x in ['/players/j/jokicni01.html'] else 0)

0      1
1      0
2      0
3      0
4      0
      ..
491    0
492    0
493    0
494    0
495    0
Name: Main Player, Length: 496, dtype: int64

In [261]:
full_df[['Posession']]

,Posession
0,Turnover by N. Jokić (bad pass; steal by A. Da...
1,Turnover by S. Hill (bad pass; steal by E. Mud...
2,Shooting foul by Ö. Aşık (drawn by J. Nurkić)
3,J. Nurkić misses free throw 1 of 2
4,Offensive rebound by Team
...,...
491,Defensive rebound by E. Mudiay
492,Personal take foul by E. Moore (drawn by E. Mu...
493,E. Mudiay makes free throw 1 of 2
494,E. Mudiay makes free throw 2 of 2


In [269]:
word_counts = {}

for val in full_df['Posession']:
    for word in val.split():
        if word.lower() in word_counts:
            word_counts[word.lower()] += 1
        else:
            word_counts[word.lower()] = 1

In [270]:
full_df['Posession'][1]

'Turnover by S. Hill (bad pass; steal by E. Mudiay)'

In [271]:
word_counts

{'turnover': 36,
 'by': 332,
 'n.': 27,
 'jokić': 25,
 '(bad': 18,
 'pass;': 14,
 'steal': 23,
 'a.': 113,
 'davis)': 24,
 's.': 24,
 'hill': 20,
 'e.': 75,
 'mudiay)': 9,
 'shooting': 21,
 'foul': 52,
 'ö.': 14,
 'aşık': 12,
 '(drawn': 51,
 'j.': 89,
 'nurkić)': 10,
 'nurkić': 39,
 'misses': 111,
 'free': 59,
 'throw': 59,
 '1': 41,
 'of': 58,
 '2': 122,
 'offensive': 33,
 'rebound': 111,
 'team': 21,
 'makes': 123,
 'moore': 21,
 '2-pt': 132,
 'jump': 99,
 'shot': 107,
 'from': 165,
 '8': 3,
 'ft': 165,
 '(assist': 49,
 'mudiay': 38,
 '(lost': 9,
 'ball;': 9,
 'moore)': 7,
 'davis': 77,
 '21': 6,
 'defensive': 83,
 'd.': 45,
 'gallinari': 24,
 'hook': 8,
 '7': 8,
 'loose': 5,
 'ball': 5,
 'aşık)': 2,
 'gallinari)': 11,
 'personal': 20,
 '15': 5,
 't.': 72,
 'frazier': 30,
 '3-pt': 43,
 '25': 20,
 'hill)': 4,
 'layup': 58,
 '26': 13,
 'jokić)': 2,
 'dunk': 10,
 'at': 10,
 'rim': 10,
 'w.': 73,
 'barton)': 6,
 '4': 3,
 '(block': 8,
 '3': 14,
 '11': 3,
 '9': 3,
 'frazier)': 19,
 '14': 1

In [272]:
sort_orders = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

for i in sort_orders:
    print(i[0], i[1])

by 332
from 165
ft 165
2-pt 132
makes 123
2 122
a. 113
misses 111
rebound 111
shot 107
jump 99
j. 89
defensive 83
davis 77
e. 75
w. 73
t. 72
free 59
throw 59
of 58
layup 58
foul 52
(drawn 51
(assist 49
k. 47
d. 45
enters 45
the 45
game 45
for 45
3-pt 43
1 41
nurkić 39
l. 39
mudiay 38
faried 37
turnover 36
offensive 33
chandler 33
barton 31
frazier 30
n. 27
jokić 25
davis) 24
s. 24
gallinari 24
steal 23
shooting 21
team 21
moore 21
hill 20
personal 20
25 20
nelson 20
stephenson 20
frazier) 19
(bad 18
b. 17
jones 17
timeout 16
pass; 14
ö. 14
3 14
hield 14
full 14
26 13
murray 13
aşık 12
galloway 12
gallinari) 11
ajinça 11
nurkić) 10
dunk 10
at 10
rim 10
faried) 10
mudiay) 9
(lost 9
ball; 9
cunningham 9
hook 8
7 8
(block 8
new 8
orleans 8
moore) 7
21 6
barton) 6
16 6
denver 6
jones) 6
loose 5
ball 5
15 5
19 5
(offensive 5
foul) 5
hill) 4
12 4
24 4
nelson) 4
stephenson) 4
pass) 4
5 4
charge 4
8 3
4 3
11 3
9 3
20 3
hield) 3
galloway) 3
chandler) 3
17 3
23 3
block 3
take 3
aşık) 2
jokić) 2
1